# Prompt caching

Prompt caching allows you to reduce overall request latency and cost for longer prompts that have identical content at the beginning of the prompt. *"Prompt"* in this context is referring to the input you send to the model as part of your chat completions request. Rather than reprocess the same input tokens over and over again, the service is able to retain a temporary cache of processed input token computations to improve overall performance. Prompt caching has no impact on the output content returned in the model response beyond a reduction in latency and cost. For supported models, cached tokens are billed at a [discount on input token pricing](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/) for Standard deployment types and up to [100% discount on input tokens](/azure/ai-services/openai/concepts/provisioned-throughput) for Provisioned deployment types.

Caches are typically cleared within **5-10** minutes of inactivity and are always removed within one hour of the cache's last use. Prompt caches are not shared between Azure subscriptions. 

## Supported models

Currently only the following models support prompt caching with Azure OpenAI:

- `o1-2024-12-17`
- `o1-preview-2024-09-12`
- `o1-mini-2024-09-12`
- `gpt-4o-2024-11-20`
- `gpt-4o-2024-08-06`
- `gpt-4o-mini-2024-07-18`
- `gpt-4o-realtime-preview` (version 2024-12-17)
- `gpt-4o-mini-realtime-preview` (version 2024-12-17)

> [!NOTE]
> Prompt caching is now also available as part of model fine-tuning for `gpt-4o` and `gpt-4o-mini`. Refer to the fine-tuning section of the [pricing page](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/) for details.

## API support

Official support for prompt caching was first added in API version `2024-10-01-preview`. At this time, only the o1 model family supports the `cached_tokens` API response parameter.

## Getting started

For a request to take advantage of prompt caching the request must be both:

- A minimum of 1,024 tokens in length.
- The first 1,024 tokens in the prompt must be identical.

When a match is found between the token computations in a prompt and the current content of the prompt cache, it's referred to as a cache hit. Cache hits will show up as [`cached_tokens`](/azure/ai-services/openai/reference-preview#cached_tokens) under [`prompt_tokens_details`](/azure/ai-services/openai/reference-preview#properties-for-prompt_tokens_details) in the chat completions response.

```json
{
  "created": 1729227448,
  "model": "o1-preview-2024-09-12",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": "fp_50cdd5dc04",
  "usage": {
    "completion_tokens": 1518,
    "prompt_tokens": 1566,
    "total_tokens": 3084,
    "completion_tokens_details": {
      "audio_tokens": null,
      "reasoning_tokens": 576
    },
    "prompt_tokens_details": {
      "audio_tokens": null,
      "cached_tokens": 1408
    }
  }
}
```

After the first 1,024 tokens cache hits will occur for every 128 additional identical tokens.

A single character difference in the first 1,024 tokens will result in a cache miss which is characterized by a `cached_tokens` value of 0. Prompt caching is enabled by default with no additional configuration needed for supported models.

## What is cached?

o1-series models feature support varies by model. For more details, see our dedicated [reasoning models guide](./reasoning.md). 

Prompt caching is supported for:

|**Caching supported**|**Description**|**Supported models**|
|--------|--------|--------|
| **Messages** | The complete messages array: system, developer, user, and assistant content | `gpt-4o`<br/>`gpt-4o-mini`<br/>`gpt-4o-realtime-preview` (version 2024-12-17)<br/>`gpt-4o-mini-realtime-preview` (version 2024-12-17)<br> `o1` (version 2024-12-17) |
| **Images** | Images included in user messages, both as links or as base64-encoded data. The detail parameter must be set the same across requests. | `gpt-4o`<br/>`gpt-4o-mini` <br> `o1` (version 2024-12-17)  |
| **Tool use** | Both the messages array and tool definitions. | `gpt-4o`<br/>`gpt-4o-mini`<br/>`gpt-4o-realtime-preview` (version 2024-12-17)<br/>`gpt-4o-mini-realtime-preview` (version 2024-12-17)<br> `o1` (version 2024-12-17) |
| **Structured outputs** | Structured output schema is appended as a prefix to the system message. | `gpt-4o`<br/>`gpt-4o-mini` <br> `o1` (version 2024-12-17) |

To improve the likelihood of cache hits occurring, you should structure your requests such that repetitive content occurs at the beginning of the messages array.

## Can I disable prompt caching?

Prompt caching is enabled by default for all supported models. There is no opt-out support for prompt caching. 


## Example 1: Caching tools and multi-turn conversations

In this example, we define tools and interactions for a customer support assistant, capable of handling tasks such as checking delivery dates, canceling orders, and updating payment methods. The assistant processes two separate messages, first responding to an initial query, followed by a delayed response to a follow-up query.

When caching tools, it is important that the tool definitions and their order remain identical for them to be included in the prompt prefix. To cache message histories in a multi-turn conversation, append new elements to the end of the messages array. In the response object and the output below, for the second completion `run2`, you can see that the `cached_tokens` value is greater than zero, indicating successful caching.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.getenv("AZURE_OPENAI_MODEl_GPT_4o_mini")

'xle-gpt-4o-mini'

In [4]:
import os
from openai import AzureOpenAI


deployment_name = 'o1-preview-2024-09-12'


client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT_GPT_o1"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY_GPT_o1"),
    api_version="2024-12-01-preview"
)

response = client.chat.completions.create(
    model=deployment_name,
    stream=False,
    messages=[
        {"role": "user", "content": "You are a helpful assistant."},
        {"role": "user", "content": "hello"}
    ]
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


In [7]:
import time
import json

# Define tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "cancel_order",
            "description": "Cancel an order that has not yet been shipped. Use this when a customer requests order cancellation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "reason": {
                        "type": "string",
                        "description": "The reason for cancelling the order."
                    }
                },
                "required": ["order_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "return_item",
            "description": "Process a return for an order. This should be called when a customer wants to return an item and the order has already been delivered.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "item_id": {
                        "type": "string",
                        "description": "The specific item ID the customer wants to return."
                    },
                    "reason": {
                        "type": "string",
                        "description": "The reason for returning the item."
                    }
                },
                "required": ["order_id", "item_id", "reason"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "update_shipping_address",
            "description": "Update the shipping address for an order that hasn't been shipped yet. Use this if the customer wants to change their delivery address.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "new_address": {
                        "type": "object",
                        "properties": {
                            "street": {
                                "type": "string",
                                "description": "The new street address."
                            },
                            "city": {
                                "type": "string",
                                "description": "The new city."
                            },
                            "state": {
                                "type": "string",
                                "description": "The new state."
                            },
                            "zip": {
                                "type": "string",
                                "description": "The new zip code."
                            },
                            "country": {
                                "type": "string",
                                "description": "The new country."
                            }
                        },
                        "required": ["street", "city", "state", "zip", "country"],
                        "additionalProperties": False
                    }
                },
                "required": ["order_id", "new_address"],
                "additionalProperties": False
            }
        }
    },
    # New tool: Update payment method
    {
        "type": "function",
        "function": {
            "name": "update_payment_method",
            "description": "Update the payment method for an order that hasn't been completed yet. Use this if the customer wants to change their payment details.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID."
                    },
                    "payment_method": {
                        "type": "object",
                        "properties": {
                            "card_number": {
                                "type": "string",
                                "description": "The new credit card number."
                            },
                            "expiry_date": {
                                "type": "string",
                                "description": "The new credit card expiry date in MM/YY format."
                            },
                            "cvv": {
                                "type": "string",
                                "description": "The new credit card CVV code."
                            }
                        },
                        "required": ["card_number", "expiry_date", "cvv"],
                        "additionalProperties": False
                    }
                },
                "required": ["order_id", "payment_method"],
                "additionalProperties": False
            }
        }
    }
]

# Enhanced system message with guardrails
messages = [
    {
        "role": "user",
        "content": """
            You are a professional, empathetic, and efficient customer support assistant. Your mission is to provide fast, clear, and comprehensive assistance to customers while maintaining a warm and approachable tone.
            
            Always express empathy, especially when the user seems frustrated or concerned, and ensure that your language is polite and professional.
            
            Use simple and clear communication to avoid any misunderstanding, and confirm actions with the user before proceeding.
            
            In more complex or time-sensitive cases, assure the user that you're taking swift action and provide regular updates.
            
            Adapt to the user’s tone: remain calm, friendly, and understanding, even in stressful or difficult situations.
            
            Additionally, there are several important guardrails that you must adhere to while assisting users:
            1. **Confidentiality and Data Privacy**: Do not share any sensitive information about the company or other users. When handling personal details such as order IDs, addresses, or payment methods, ensure that the information is treated with the highest confidentiality. If a user requests access to their data, only provide the necessary information relevant to their request, ensuring no other user's information is accidentally revealed.
            2. **Secure Payment Handling**: When updating payment details or processing refunds, always ensure that payment data such as credit card numbers, CVVs, and expiration dates are transmitted and stored securely. Never display or log full credit card numbers. Confirm with the user before processing any payment changes or refunds.
            3. **Respect Boundaries**: If a user expresses frustration or dissatisfaction, remain calm and empathetic but avoid overstepping professional boundaries. Do not make personal judgments, and refrain from using language that might escalate the situation. Stick to factual information and clear solutions to resolve the user's concerns.
            4. **Legal Compliance**: Ensure that all actions you take comply with legal and regulatory standards. For example, if the user requests a refund, cancellation, or return, follow the company’s refund policies strictly. If the order cannot be canceled due to being shipped or another restriction, explain the policy clearly but sympathetically.
            5. **Consistency**: Always provide consistent information that aligns with company policies. If unsure about a company policy, communicate clearly with the user, letting them know that you are verifying the information, and avoid providing false promises. If escalating an issue to another team, inform the user and provide a realistic timeline for when they can expect a resolution.
            6. **User Empowerment**: Whenever possible, empower the user to make informed decisions. Provide them with relevant options and explain each clearly, ensuring that they understand the consequences of each choice (e.g., canceling an order may result in loss of loyalty points, etc.). Ensure that your assistance supports their autonomy.
            7. **No Speculative Information**: Maintain strict adherence to factual information and verified data when communicating with users. Under no circumstances should you engage in speculation about potential outcomes or share information that hasn't been thoroughly verified. When discussing order statuses, company policies, or possible solutions to customer issues, ensure that every piece of information provided is backed by concrete evidence and official documentation. In situations where certainty is lacking or information requires verification, clearly communicate to the user that additional investigation is needed before any definitive statements or commitments can be made. This approach helps maintain trust, prevents misunderstandings, and ensures that users receive only accurate, reliable information they can confidently act upon. If faced with uncertainty about any aspect of a customer inquiry, take the time to thoroughly research and verify the information rather than making assumptions or providing tentative answers.
            8. **Respectful and Inclusive Language**: Ensure that your language remains consistently inclusive, respectful, and culturally sensitive in all interactions, regardless of the user's tone or communication style. Take special care to use gender-neutral language and avoid cultural, racial, or socioeconomic assumptions. Be mindful of diverse user needs, backgrounds, and accessibility requirements, adapting your communication style appropriately while maintaining professionalism. Consider regional and cultural differences in communication preferences, and always err on the side of being more formal until the user indicates a preference for a more casual interaction style.
            9. **Respectful and Inclusive Language**: Ensure that your language remains consistently inclusive, respectful, and culturally sensitive in all interactions, regardless of the user's tone or communication style. Take special care to use gender-neutral language and avoid cultural, racial, or socioeconomic assumptions. Be mindful of diverse user needs, backgrounds, and accessibility requirements, adapting your communication style appropriately while maintaining professionalism. Consider regional and cultural differences in communication preferences, and always err on the side of being more formal until the user indicates a preference for a more casual interaction style.
            10. **Respectful and Inclusive Language**: Ensure that your language remains consistently inclusive, respectful, and culturally sensitive in all interactions, regardless of the user's tone or communication style. Take special care to use gender-neutral language and avoid cultural, racial, or socioeconomic assumptions. Be mindful of diverse user needs, backgrounds, and accessibility requirements, adapting your communication style appropriately while maintaining professionalism. Consider regional and cultural differences in communication preferences, and always err on the side of being more formal until the user indicates a preference for a more casual interaction style.
            11. **Respectful and Inclusive Language**: Ensure that your language remains consistently inclusive, respectful, and culturally sensitive in all interactions, regardless of the user's tone or communication style. Take special care to use gender-neutral language and avoid cultural, racial, or socioeconomic assumptions. Be mindful of diverse user needs, backgrounds, and accessibility requirements, adapting your communication style appropriately while maintaining professionalism. Consider regional and cultural differences in communication preferences, and always err on the side of being more formal until the user indicates a preference for a more casual interaction style.
            12. **Respectful and Inclusive Language**: Ensure that your language remains consistently inclusive, respectful, and culturally sensitive in all interactions, regardless of the user's tone or communication style. Take special care to use gender-neutral language and avoid cultural, racial, or socioeconomic assumptions. Be mindful of diverse user needs, backgrounds, and accessibility requirements, adapting your communication style appropriately while maintaining professionalism. Consider regional and cultural differences in communication preferences, and always err on the side of being more formal until the user indicates a preference for a more casual interaction style.
        """
    },
    {
        "role": "user",
        "content": """
            Hi, I placed an order three days ago and haven’t received any updates on when it’s going to be delivered.
            Could you help me check the delivery date? My order number is #9876543210. I’m a little worried because I need this item urgently.
        """
    }
]

# Enhanced user_query2
user_query2 = {
    "role": "user",
    "content": (
        "Since my order hasn't actually shipped yet, I would like to cancel it. "
        "The order number is #9876543210, and I need to cancel because I’ve decided to purchase it locally to get it faster. "
        "Can you help me with that? Thank you!"
    )
}


# Function to run completion with the provided message history and tools
def completion_run(messages, tools):
    completion = client.chat.completions.create(
        model=deployment_name,
        # tools=tools,
        messages=messages,
        # tool_choice="required"
    )
    usage_data = json.dumps(completion.to_dict(), indent=4)
    return usage_data


# Main function to handle the two runs
def main(messages, tools, user_query2):
    # Run 1: Initial query
    print("Run 1:")
    run1 = completion_run(messages, tools)
    print(run1)

    # Delay for 7 seconds
    time.sleep(7)

    # Append user_query2 to the message history
    messages.append(user_query2)

    # Run 2: With appended query
    print("\nRun 2:")
    run2 = completion_run(messages, tools)
    print(run2)


# Run the main function
main(messages, tools, user_query2)

Run 1:
{
    "id": "chatcmpl-B0GEvhNNqrLVag52OE3nb2LMQdfE5",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "message": {
                "content": "Hello,\n\nThank you for reaching out, and I'm sorry to hear that you haven't received any updates on your order. I understand how important it is for you to receive your item urgently, and I'm here to assist you.\n\nLet me check the status of your order **#9876543210** for you.\n\n*\\[Brief pause as if checking the system\\]*\n\nThank you for your patience. It appears that your order is currently being processed. I apologize for the delay and any inconvenience this may have caused.\n\nTo help ensure you receive your item as soon as possible, I can request our shipping department to prioritize your order. Would you like me to proceed with this?\n\nPlease let me know if there's anything else I can assist you with.",
                "role": "assistant"
            },
            "content_fi

In [ ]:
sauce_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/12-04-20-saucen-by-RalfR-15.jpg/800px-12-04-20-saucen-by-RalfR-15.jpg"
veggie_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Veggies.jpg/800px-Veggies.jpg"
eggs_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Egg_shelf.jpg/450px-Egg_shelf.jpg"
milk_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Lactaid_brand.jpg/800px-Lactaid_brand.jpg"


def multiimage_completion(url1, url2, user_query):
    completion = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": url1,
                            "detail": "high"
                        },
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": url2,
                            "detail": "high"
                        },
                    },
                    {"type": "text", "text": user_query}
                ],
            }
        ],
        max_tokens=300,
    )
    print(json.dumps(completion.to_dict(), indent=4))


def main(sauce_url, veggie_url):
    multiimage_completion(
        sauce_url, veggie_url, "Please list the types of sauces are shown in these images")
    # delay for 20 seconds
    time.sleep(20)
    multiimage_completion(
        sauce_url, veggie_url, "Please list the types of vegetables are shown in these images")
    time.sleep(20)
    multiimage_completion(
        milk_url, sauce_url, "Please list the types of sauces are shown in these images")


main(sauce_url, veggie_url)

## Tips

Here are the essential steps for prompt caching:

- Put your static and repeated content first in prompts and dynamic content last. This simple order makes the cache work better.
- Use prompts consistently. Frequent usage keeps prompts in the cache, while infrequent usage leads to automatic removal.
- Check your metrics regularly. Look at how often the cache is used, how fast it runs, and how much content is cached so you can make it work better.

These practices will help you get the most out of prompt caching. Using an effective cache makes your apps run faster and reduces costs, leading to better performance for users.